<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('wordnet')
import re
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# remove unwanted chars
# convert to lowercase
# remove unwanted spaces
# remove stop words
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 

def preprocess_text(corpus, text_lower_case=True, 
                      special_char_removal=True, stopword_removal=True, remove_digits=False):    
    normalized_text = []
    # normalize each document in the corpus
    for doc in corpus:
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits) 

        if stopword_removal:
            doc = remove_stopwords(doc)

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)            
        normalized_text.append(doc)
        
    return normalized_text

def remove_special_characters(text, remove_digits=False):
    #Using regex
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text):  
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]   
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)                 
    return ' '.join(filtered_sentence)

In [49]:
from sklearn.model_selection import train_test_split

squad_df = pd.read_csv(project_path+'squad_data_final.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)
squad_df.tail(5)

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer
130301,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,NaN,NaN,485.0,matter,True,term matter used throughout physics bewilderin...,physics has broadly agreed on the definition o...,IMPOSSIBLE
130302,Matter,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,5a7e070b70df9f001a87543a,NaN,NaN,327.0,Alfvén,True,term matter used throughout physics bewilderin...,who coined the term partonic matter,IMPOSSIBLE
130303,Matter,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,5a7e070b70df9f001a87543b,NaN,NaN,350.0,Gk. common matter,True,term matter used throughout physics bewilderin...,what is another name for antimatter,IMPOSSIBLE
130304,Matter,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,NaN,NaN,529.0,a specifying modifier,True,term matter used throughout physics bewilderin...,matter usually does not need to be used in con...,IMPOSSIBLE
130305,Matter,"The term ""matter"" is used throughout physics i...",What field of study has a variety of unusual c...,5a7e070b70df9f001a87543d,NaN,NaN,37.0,physics,True,term matter used throughout physics bewilderin...,what field of study has a variety of unusual c...,IMPOSSIBLE


In [50]:
squad_df[squad_df['id'] == '5730f1c3e6313a140071cad6']

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer


In [0]:
Xcontext_train, Xcontext_test, yquestion_train, yquestion_test = train_test_split(squad_df['clean_context'], squad_df['clean_question'], test_size=0.33, random_state=42)
Xquestion_train, Xquestion_test, yanswer_train, yanswer_test = train_test_split(squad_df['clean_question'], squad_df['clean_answer'], test_size=0.33, random_state=42)

In [52]:
print(Xcontext_train.shape)
print(Xcontext_test.shape)
print(yquestion_train.shape)
print(yquestion_test.shape)

(87305,)
(43001,)
(87305,)
(43001,)


In [53]:
# Define TF-ID Venctorize and Label encoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

xvectorizer = TfidfVectorizer(max_features=5000)
xvectorizer.fit(squad_df['clean_context'] + " " + squad_df['clean_question'])
ylblencoder = LabelEncoder()
# # generate label encoder for both question and clean answer
ylblencoder.fit(squad_df['clean_question'].append(squad_df['clean_answer']).reset_index(drop=True).astype(str))

LabelEncoder()

In [54]:
print(len(ylblencoder.classes_))
len(xvectorizer.vocabulary_)

193496


5000

In [0]:
# Trim to top 2000 only
size = 10000
Xcontext_vectorized = xvectorizer.transform(Xcontext_train.head(size))
yquestion_vectorized = ylblencoder.transform(yquestion_train.head(size))

Xquestion_vectorized = xvectorizer.transform(Xquestion_train.head(size))
yanswer_vectorized = ylblencoder.transform(yanswer_train.head(size))

In [0]:
from sklearn import svm

# SVM model 1

context2question = svm.SVC()    
context2question.fit(x_vectorized,y_vectorized)

question2answer = svm.SVC(gamma=0.025, C=10)    
question2answer.fit(xvectorizer.transform(Xquestion_train),ylblencoder.transform(yanswer_train))

print('Model context2question', context2question)
print('Model question2answer', question2answer)